In [1]:
import numpy as np
import numpy.random as rn
import pandas as pd
from sklearn.model_selection import KFold    # 获得k个折数
import matplotlib.pylab as plt

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# win
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

np.power(2, 3)

np.power(2, 1 / 2)

1.4142135623730951

In [201]:
def f_lambda(A, x, y, mu, lambda_0, thresholding_value):
    """
    功能：希望给定xi---> b_mu---->f_lamda_mu
    :param xi:
    :return:
    """
    PI = 3.141493
    n = x.shape[0]
    x_trans = [0] * n  # 初始化计算所得的矩阵为0
    b_mu = x + mu * np.dot(A.T, y - np.dot(A, x))
    for i in range(n):
        if abs(b_mu[i]) > thresholding_value:
            val_i = (lambda_0 / 8) * np.abs(b_mu[i] / 3)**(
                -1.5)  # np.power(a, b) ,notice b >0
            if np.abs(val_i) > 1:  #防止val_i的值大于1，使得arccos报错， 进行最大最小标准化
                val_i = (val_i - thresholding_value) / val_i
            ph_i = np.arccos(val_i)
            x_trans[i] = 2 / 3 * b_mu[i] * (1 + np.cos(2 / 3 * (PI - ph_i)))
    return np.array(x_trans)


if __name__ == '__main__':
    n, m = 200, 20
    A = rn.randn(n, m)
    X = rn.randn(m, )
    y = np.dot(A, X) + rn.randn(n, )
    mu = 1 / np.sum(A**2)
    thresholding_value = np.power(54, 1 / 3) / 4 * np.power(lambda_0 * mu, 2 / 3)
    x0 = rn.randn(m, )
    lambda_0 = 0.1
    x = f_lambda(A, x0, y, mu, lambda_0, thresholding_value)
    print(x)
    

[ 0.49407633 -0.22669128 -0.57490486  0.22703182 -1.3922278  -0.48682657
 -0.46270689  1.71694907  0.33111924 -1.01828664  1.11704846 -1.56487147
  0.3724485   0.8516109   0.42451952  0.15359524 -0.80094375  2.62421094
  0.44428154 -0.42323924]


In [206]:
# 最小损失函数的定义：

def loss_value(A, x, y, lambda_0):
    """损失函数定义"""
    loss = np.sum((y - np.dot(A, x0)) ** 2)
    penal = lambda_0 * np.sum(np.abs(x0) ** 0.5)         # x0可能有负数
    return loss + penal
    

def loss_function(A, x0, y, mu, lambda_0, max_iter = 500, tol = 1e-8):
    """
    :param A: 
    :param x0: x的初始化
    :param y: 
    :param mu: 超参数
    :param lambda_0: 超参数
    :param max_iter: 最大迭代次数
    :param tol: 容忍度
    :return: 
    """
    best_loss = np.inf
    thresholding_value = np.power(54, 1 / 3) / 4 * np.power(lambda_0 * mu, 2 / 3)
    for i in range(max_iter):
        x0 = f_lambda(A, x0, y, mu, lambda_0, thresholding_value)
        loss = loss_value(A, x0, y, lambda_0)
        if (abs(best_loss - loss) < tol) and (i > 5):
            best_loss = loss
            break
        if loss < best_loss:
            best_loss = loss
    return x0, best_loss
        
    
if __name__ == '__main__':
    rn.seed(2211)
    n, m = 200, 20
    A = rn.randn(n, m)
    X = np.array([1,2,3,4,5,0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    y = np.dot(A, X) + rn.randn(n,)
    x0 = rn.randn(m,)
    mu = 1 / np.sum(A**2)
    lambda_0 = 0.2
    x_best, loss= loss_function(A, x0, y, mu, lambda_0)
    print('x_best: \n{} \n loss\n{}'.format(x_best, loss))
#     x_best = loss_function(A, x0, y, mu, lambda_0)[0]
#     print(x_best)
    



x_best: 
[-5.53215624e-02  4.75490425e-02  1.38273847e+00  7.08932886e-02
  1.73869260e+00  2.00932934e-01  8.40146864e-04 -3.09892678e-01
  1.55349653e-02 -8.10946034e-01 -6.60557141e-01  4.02767014e-01
 -9.65873306e-01  1.49158554e-02  7.62520540e-01 -2.78668744e-02
 -4.97306203e-04 -3.16660607e-01 -9.91581874e-01  3.06028088e-01] 
 loss
16435.708346105872


In [210]:
# 选择最优的lambda_0

def select_best_lambda(A, y, x0, mu):
    """
    :param A: 
    :param y: 
    :param x: 
    :param mu: 
    :param max_iter: 
    :param tol: 
    :return: best_lambda
    """
    # cross valication
    loss_fun_all = [] # 
    lambda_all = np.arange(1, 100) * 0.01
    n = lambda_all.shape[0]
    sk_fold = KFold(n_splits= 10, shuffle=True, random_state=111)
    for i in range(n):
        loss_temp_test = np.zeros(n)   # 保存每个lambda_all[i]对应的loss_test
        for train_index, dev_index in sk_fold.split(A, y):
            x_train = loss_function(A[train_index], x0, y[train_index], mu = mu, lambda_0 = lambda_all[i], max_iter = 100, tol = 1e-5)[0] # 只需记录每一次的x_0的值用于计算预测的损失
            loss_temp_test[i] = np.mean(loss_value(A[dev_index], x_train, y[dev_index], lambda_all[i]))   # 计算出每次
        loss_fun_all.append([lambda_all[i], np.mean(loss_temp_test)])
    loss_fun_all = np.array(loss_fun_all)
#     print(loss_fun_all)
    min_index = np.argmin(loss_fun_all[:,1])
    lambda_best = loss_fun_all[min_index][0]
    return lambda_best


        
if __name__ == '__main__':
    rn.seed(2211)
    n, m = 200, 20
    A = rn.randn(n, m)
    X = np.array([10, 9, 3,4,5,0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    y = np.dot(A, X) + rn.randn(n)
    x0 = rn.randn(m)
    mu = 1 / np.sum(A**2)
    lambda_0 = select_best_lambda(A, y, x0, mu=mu)
    print(lambda_0)
    
    
    

0.01


In [153]:
# 这个用于所有超参数都训练完成了

# from sklearn.model_selection import train_test_split

# rn.seed(22)
# n, m = 10, 2
# A = rn.randn(n, m)
# A
# X = rn.rand(m)
# y = np.dot(A, X) + rn.randn(n, )
# y 
# #x为数据集的feature熟悉，y为label.
# x_train, x_test, y_train, y_test = train_test_split(A, y, test_size = 0.3)
# x_train, x_test, y_train, y_test 

In [169]:
sk_fold = KFold(n_splits= 10, shuffle=True, random_state=111)
for train_index, dev_index in sk_fold.split(A, y):
        print( "TRAIN:", train_index, "TEST:", dev_index)
#         print(A[train_index])

TRAIN: [ 1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 29 31 32 33 34 35 36 37 38 39] TEST: [ 0  9 28 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 16 17 18 19 20 21 22 23 24 25
 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [11 15 26 27]
TRAIN: [ 0  1  2  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [ 3  4 17 23]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 32 34 35 38 39] TEST: [31 33 36 37]
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 30 31 32 33 34 36 37 38 39] TEST: [ 6 13 29 35]
TRAIN: [ 0  1  2  3  4  6  7  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26
 27 28 29 30 31 32 33 35 36 37 38 39] TEST: [ 5  8 21 34]
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 15 17 18 19 20 21 22 23 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [ 1 14 16 24]
TRAIN: [ 0  1  2  3  4  5  6  7  8

In [3]:
rn.seed(2211)
n, m = 2000, 20
A = rn.randn(n, m)
X = np.array([10, 90, 30,40,50,0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0])
y = np.dot(A, X) + rn.randn(n)
print(y)
x0 = rn.randn(m)
mu = 1 / np.sum(A**2)

[   4.75965596    7.85835749  135.3870095  ... -165.04792203  -87.58024323
  -69.58501366]
